In [1]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import itertools as it
import tqdm
import pandas as pd

In [2]:
ROOT_DIR = os.path.abspath("__file__" + "/../../")
sys.path.insert(0, f"{ROOT_DIR}")

In [3]:
spin_j = 3
env_name = f"single vertex spinfoam/j={float(spin_j)}"

mcmc_chains = np.load(f"{ROOT_DIR}/data/MCMC/{env_name}/mcmc_chains.npy")
# mcmc_masks = np.load(f"{ROOT_DIR}/data/MCMC/{env_name}/acceptance_masks.npy")
# gfn_states = np.load(f"{ROOT_DIR}/data/GFN/{env_name}/terminal_states.npy")

In [4]:
def get_distributions_over_time(grid_positions, grid_len, every_n_iterations=100, masks=None):
    n_iterations, _, grid_dim = grid_positions.shape

    counts = np.zeros(shape=(grid_len, )*grid_dim)
    n_samples = 0
    empirical_distributions_over_time = []
    n_samples_used_over_time = []
    for i in range(0, n_iterations, every_n_iterations):
        if masks is not None:
            selected_positions = [
                pos[mask] for pos, mask in zip(
                    grid_positions[i:i+every_n_iterations], masks[i:i+every_n_iterations]
                )
            ]
        else:
            selected_positions = grid_positions[i:i+every_n_iterations]
            
        states = np.concatenate(selected_positions)

        n_samples += states.shape[0]
        np.add.at(counts, tuple(states.T), 1)
        
        empirical_distributions_over_time.append(counts/n_samples)
        n_samples_used_over_time.append(n_samples)
    empirical_distributions_over_time = np.stack(empirical_distributions_over_time)
    return empirical_distributions_over_time, n_samples_used_over_time

def get_distributions_coordinates_over_time(grid_len, distributions_over_time, iteration = -1):
    result = []
    if len(distributions_over_time.shape) == 6:
        for dist in tqdm.tqdm(distributions_over_time, total=distributions_over_time.shape[0]):
            s = (grid_len for _ in range(grid_len))
            data = np.array(
                    list(np.ndindex(*s))
                )
            data = np.apply_along_axis(lambda x: (x[0], x[1], x[2], x[3], x[4], dist[x[0], x[1], x[2], x[3], x[4]]), 1, data)
            result.append(data)
        # distributions.append([i1, i2, i3, i4, i5, distributions_over_time[iteration, i1, i2, i3, i4, i5]])
    elif len(distributions_over_time.shape) == 5:
        ...
        # distributions.append([i1, i2, i3, i4, i5, distributions_over_time[i1, i2, i3, i4, i5]])

    return np.stack(data, axis=1)

In [5]:
grid_len = int(2*spin_j + 1)

mcmc_distributions_over_time, mcmc_n_t = get_distributions_over_time(
    mcmc_chains, grid_len, every_n_iterations=10000, 
#     masks=mcmc_masks
)
# gfn_distributions_over_time, gfn_n_t = get_distributions_over_time(
#     gfn_states, grid_len, every_n_iterations=100
# )

In [6]:
# Create a 2D array of the grid coordinates and the respective rewards, flattened.
# gfn = get_distributions_coordinates_over_time(grid_len, gfn_distributions_over_time)

mcmc = get_distributions_coordinates_over_time(grid_len, mcmc_distributions_over_time)

100%|██████████| 10/10 [00:54<00:00,  5.42s/it]


In [7]:
mcmc.shape

(6, 823543)

In [8]:
grid_len = 8
s = (grid_len for _ in range(grid_len))
df = np.array(
        list(np.ndindex(*s))
    )
# .reshape(*s, -1)